# 입문자를 위한, 파이썬/R 데이터 분석      

]

##  (공공데이터분석) 국토부 아파트매매 실거래가 API[data.go.kr] 연동

#### XML 국토교통부_아파트매매 실거래자료 : https://www.data.go.kr/data/15058747/openapi.do

#### 행정표준관리시스템 : https://www.code.go.kr/index.do ( [메뉴] 자주이용하는 코드 > 법정동 > [법정동코드전체자료] )

]

### 1. Import Packages

In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### 2. Data Loading 

#### : API Key Setting

In [9]:
f=open("data/my_api_key.txt","r")
lines=f.readlines()
my_key=lines[0]
f.close()

In [10]:
my_key

'QkiT9isHn%2FngYj7SJNBftK290c7SSWktGzgux4c8T%2Bu5%2BHox2jrUAqrY%2BdIjzM9XMSYrqLuzpU5V%2BvqTyvIs8Q%3D%3D'

#### : Pre-defined

In [11]:
# "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade
#        ?
#        LAWD_CD=11110
#        &
#        DEAL_YMD=201512
#        &
#        serviceKey=서비스키"

In [12]:
base_url = "http://apis.data.go.kr/B552061/AccidentDeath/getRestTrafficAccidentDeath?ServiceKey=QkiT9isHn%2FngYj7SJNBftK290c7SSWktGzgux4c8T%2Bu5%2BHox2jrUAqrY%2BdIjzM9XMSYrqLuzpU5V%2BvqTyvIs8Q%3D%3D&searchYear=2019&siDo=1100&guGun=1116&numOfRows=20&pageNo=1"

In [13]:
area_cd = "LAWD_CD=" + "11680"    # 11680 : 강남구
deal_dt = "DEAL_YMD=" + "202306"
svc_key = "serviceKey=" + my_key

In [14]:
url = base_url + "?" + si_cd + "&" + area_cd + "&" + svc_key
url

NameError: name 'si_cd' is not defined

#### : Request Test

In [15]:
response = requests.get(url)
response
# response 200은 정상적!!

NameError: name 'url' is not defined

In [16]:
response

NameError: name 'response' is not defined

### Addtional Preparation : 

#### -1)  Input Data : lawd_cd / 법정동 코드(지역코드) 가공

In [17]:
lawd_cd = pd.read_csv("data/법정동코드 전체자료.txt", sep='\t', encoding ='cp949') #'euc-kr', 'utf-8'
lawd_cd

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
49845,5280042024,전북특별자치도 부안군 위도면 대리,존재
49846,5280042025,전북특별자치도 부안군 위도면 거륜리,존재
49847,5280042026,전북특별자치도 부안군 위도면 식도리,존재
49848,5280042027,전북특별자치도 부안군 위도면 상왕등리,존재


In [18]:
# Column 이름 변경 : 데이터 조작 편의성을 위함
print(lawd_cd.columns)

Index(['법정동코드', '법정동명', '폐지여부'], dtype='object')


In [19]:
lawd_cd.columns = ["code", "name", "check"]
lawd_cd

,code,name,check
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
49845,5280042024,전북특별자치도 부안군 위도면 대리,존재
49846,5280042025,전북특별자치도 부안군 위도면 거륜리,존재
49847,5280042026,전북특별자치도 부안군 위도면 식도리,존재
49848,5280042027,전북특별자치도 부안군 위도면 상왕등리,존재


In [22]:
lawd_cd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49850 entries, 0 to 49849
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    49850 non-null  int64 
 1   name    49850 non-null  object
 2   check   49850 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [26]:
in_name = "종로구"

In [27]:
# lawd_cd['name'].str.contains(in_name)
# lawd_cd["name"].str.contains(in_name)
lawd_cd["name"].str.contains(in_name)

0        False
1         True
2         True
3         True
4         True
         ...  
49845    False
49846    False
49847    False
49848    False
49849    False
Name: name, Length: 49850, dtype: bool

In [28]:
# lawd_cd['check'] == "존재"
lawd_cd["name"] == "폐지"

0        False
1        False
2        False
3        False
4        False
         ...  
49845    False
49846    False
49847    False
49848    False
49849    False
Name: name, Length: 49850, dtype: bool

In [29]:
# lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")]
lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] =="존재")]

,code,name,check
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
5,1111010400,서울특별시 종로구 효자동,존재
...,...,...,...
84,1111018300,서울특별시 종로구 평창동,존재
85,1111018400,서울특별시 종로구 부암동,존재
86,1111018500,서울특별시 종로구 홍지동,존재
87,1111018600,서울특별시 종로구 신영동,존재


In [30]:
# tmp = lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")].head(1)
tmp = lawd_cd[(lawd_cd["name"].str.contains(in_name)) & (lawd_cd["check"] == "존재")].head(1)
tmp

,code,name,check
1,1111000000,서울특별시 종로구,존재


In [31]:
tmp.loc[1,'code'] # 가져오기는 하나 문제
                  

1111000000

In [32]:
tmp.iloc[0,0] # 인덱스로 데이터를 가져오도록
              

1111000000

In [33]:
tmp = tmp.iloc[0,0]
str(tmp)
tmp = tmp.astype(str)
tmp[0:5]

'11110'

In [34]:
lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]

'11110'

In [35]:
# Function 작성 :
def find_lawd_cd(in_text):
    return lawd_cd[(lawd_cd['name'].str.contains(in_text)) & (lawd_cd['check'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]    

In [36]:
# in_name = "구로구"
# in_name = "강남구"
# in_name = "영등포구"
# in_name = "종로구"
find_lawd_cd(in_name)

'11110'

#### -2)  Input Data : lawd_cd, deal_ymd 입력받기

In [38]:
in_lawd = input("검색지역 입력 : ")

검색지역 입력 : 종로구


In [39]:
in_lawd

'종로구'

In [32]:
in_deal_ymd = input("계약월 입력(예: 201912) : ")

계약월 입력(예: 201912) : 202402


In [33]:
in_deal_ymd

'202402'

In [34]:
in_svckey = my_key

In [35]:
in_lawd_cd = find_lawd_cd(in_lawd)
in_lawd_cd

'11680'

In [36]:
area_cd = "LAWD_CD=" + in_lawd_cd
deal_dt = "DEAL_YMD=" + in_deal_ymd
svc_key = "serviceKey=" + in_svckey

In [37]:
url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key
url

'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=11680&DEAL_YMD=202402&serviceKey=QkiT9isHn%2FngYj7SJNBftK290c7SSWktGzgux4c8T%2Bu5%2BHox2jrUAqrY%2BdIjzM9XMSYrqLuzpU5V%2BvqTyvIs8Q%3D%3D'

In [38]:
response = requests.get(url)
response

<Response [200]>

In [39]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><거래금액>     233,000</거래금액><거래유형>중개거래</거래유형><건축년도>2006</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>역삼푸르지오</아파트><월>2</월><일>2</일><전용면적>84.9097</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>754-1</지번><지역코드>11680</지역코드><층>7</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>     197,500</거래금액><거래유형>중개거래</거래유형><건축년도>2005</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>e-편한세상</아파트><월>2</월><일>2</일><전용면적>59.606</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>755-4</지번><지역코드>11680</지역코드><층>25</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>      69,000</거래금액><거래유형>중개거래</거래유형><건축년도>2006</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>역삼I\'PARK</아파트><월>2</월><일>3</일><전용면적>28.246</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>713-11</지번><지역코드>11680</지역

### 3. Data Preprocessng 

#### : web data / API output

In [40]:
soup = BeautifulSoup(response.text, "lxml")
soup

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><response><header><resultcode>00</resultcode><resultmsg>NORMAL SERVICE.</resultmsg></header><items><item>&lt;거래금액&gt;     233,000거래금액&gt;&lt;거래유형&gt;중개거래거래유형&gt;&lt;건축년도&gt;2006건축년도&gt;&lt;년&gt;2024년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt;개인매도자&gt;&lt;매수자&gt;개인매수자&gt;&lt;법정동&gt; 역삼동법정동&gt;&lt;아파트&gt;역삼푸르지오아파트&gt;&lt;월&gt;2월&gt;&lt;일&gt;2일&gt;&lt;전용면적&gt;84.9097전용면적&gt;&lt;중개사소재지&gt;서울 강남구중개사소재지&gt;&lt;지번&gt;754-1지번&gt;&lt;지역코드&gt;11680지역코드&gt;&lt;층&gt;7층&gt;&lt;해제사유발생일&gt; 해제사유발생일&gt;&lt;해제여부&gt; 해제여부&gt;</item><item>&lt;거래금액&gt;     197,500거래금액&gt;&lt;거래유형&gt;중개거래거래유형&gt;&lt;건축년도&gt;2005건축년도&gt;&lt;년&gt;2024년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt;개인매도자&gt;&lt;매수자&gt;개인매수자&gt;&lt;법정동&gt; 역삼동법정동&gt;&lt;아파트&gt;e-편한세상아파트&gt;&lt;월&gt;2월&gt;&lt;일&gt;2일&gt;&lt;전용면적&gt;59.606전용면적&gt;&lt;중개사소재지&gt;서울 강남구중개사소재지&gt;&lt;지번&gt;755-4지번&gt;&lt;지역코드&gt;11680지역코드&gt;&lt;층&gt;25층&gt;&lt;해제사유발생일&gt; 해제사유발생일&gt;&lt;

In [41]:
soup.find_all('item')

[<item>&lt;거래금액&gt;     233,000거래금액&gt;&lt;거래유형&gt;중개거래거래유형&gt;&lt;건축년도&gt;2006건축년도&gt;&lt;년&gt;2024년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt;개인매도자&gt;&lt;매수자&gt;개인매수자&gt;&lt;법정동&gt; 역삼동법정동&gt;&lt;아파트&gt;역삼푸르지오아파트&gt;&lt;월&gt;2월&gt;&lt;일&gt;2일&gt;&lt;전용면적&gt;84.9097전용면적&gt;&lt;중개사소재지&gt;서울 강남구중개사소재지&gt;&lt;지번&gt;754-1지번&gt;&lt;지역코드&gt;11680지역코드&gt;&lt;층&gt;7층&gt;&lt;해제사유발생일&gt; 해제사유발생일&gt;&lt;해제여부&gt; 해제여부&gt;</item>,
 <item>&lt;거래금액&gt;     197,500거래금액&gt;&lt;거래유형&gt;중개거래거래유형&gt;&lt;건축년도&gt;2005건축년도&gt;&lt;년&gt;2024년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt;개인매도자&gt;&lt;매수자&gt;개인매수자&gt;&lt;법정동&gt; 역삼동법정동&gt;&lt;아파트&gt;e-편한세상아파트&gt;&lt;월&gt;2월&gt;&lt;일&gt;2일&gt;&lt;전용면적&gt;59.606전용면적&gt;&lt;중개사소재지&gt;서울 강남구중개사소재지&gt;&lt;지번&gt;755-4지번&gt;&lt;지역코드&gt;11680지역코드&gt;&lt;층&gt;25층&gt;&lt;해제사유발생일&gt; 해제사유발생일&gt;&lt;해제여부&gt; 해제여부&gt;</item>,
 <item>&lt;거래금액&gt;      69,000거래금액&gt;&lt;거래유형&gt;중개거래거래유형&gt;&lt;건축년도&gt;2006건축년도&gt;&lt;년&gt;2024년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일

In [52]:
item = soup.find('item')
item

<item>&lt;거래금액&gt;     233,000거래금액&gt;&lt;거래유형&gt;중개거래거래유형&gt;&lt;건축년도&gt;2006건축년도&gt;&lt;년&gt;2024년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt;개인매도자&gt;&lt;매수자&gt;개인매수자&gt;&lt;법정동&gt; 역삼동법정동&gt;&lt;아파트&gt;역삼푸르지오아파트&gt;&lt;월&gt;2월&gt;&lt;일&gt;2일&gt;&lt;전용면적&gt;84.9097전용면적&gt;&lt;중개사소재지&gt;서울 강남구중개사소재지&gt;&lt;지번&gt;754-1지번&gt;&lt;지역코드&gt;11680지역코드&gt;&lt;층&gt;7층&gt;&lt;해제사유발생일&gt; 해제사유발생일&gt;&lt;해제여부&gt; 해제여부&gt;</item>

In [53]:
#item.find('거래금액').text

In [54]:
import xml.etree.ElementTree as ET

In [55]:
response#.content#.text

<Response [200]>

In [56]:
root = ET.fromstring(response.content)
root

<Element 'response' at 0x000001D5062F5130>

In [57]:
item_list = []

In [58]:
root.find('body').find('items')[0]

<Element 'item' at 0x000001D5062F5360>

In [59]:
type(root.find('body').find('items'))

xml.etree.ElementTree.Element

In [60]:
child = root.find('body').find('items')[0]
child.findall('*')#[0]

[<Element '거래금액' at 0x000001D5062F5400>,
 <Element '거래유형' at 0x000001D5062F5450>,
 <Element '건축년도' at 0x000001D5062F54F0>,
 <Element '년' at 0x000001D5062F55E0>,
 <Element '동' at 0x000001D5062F56D0>,
 <Element '등기일자' at 0x000001D5062F5720>,
 <Element '매도자' at 0x000001D5062F5770>,
 <Element '매수자' at 0x000001D5062F57C0>,
 <Element '법정동' at 0x000001D5062F5810>,
 <Element '아파트' at 0x000001D5062F58B0>,
 <Element '월' at 0x000001D5062F5900>,
 <Element '일' at 0x000001D5062F59A0>,
 <Element '전용면적' at 0x000001D5062F59F0>,
 <Element '중개사소재지' at 0x000001D5062F5A40>,
 <Element '지번' at 0x000001D5062F5A90>,
 <Element '지역코드' at 0x000001D5062F5AE0>,
 <Element '층' at 0x000001D5062F5B80>,
 <Element '해제사유발생일' at 0x000001D5062F5BD0>,
 <Element '해제여부' at 0x000001D5062F5C20>]

In [61]:
element = child.findall('*')[0]

In [62]:
element.tag.strip()

'거래금액'

In [63]:
element.text.strip()

'233,000'

In [64]:
data = {}

In [65]:
tag = element.tag.strip()
text = element.text.strip()
data[tag] = text

In [66]:
data

{'거래금액': '233,000'}

In [67]:
def find_xml_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            # print tag, text
            data[tag] = text
        item_list.append(data)  
    return item_list

In [68]:
items_list = find_xml_items(response)
items_list

[{'거래금액': '233,000',
  '거래유형': '중개거래',
  '건축년도': '2006',
  '년': '2024',
  '동': '',
  '등기일자': '',
  '매도자': '개인',
  '매수자': '개인',
  '법정동': '역삼동',
  '아파트': '역삼푸르지오',
  '월': '2',
  '일': '2',
  '전용면적': '84.9097',
  '중개사소재지': '서울 강남구',
  '지번': '754-1',
  '지역코드': '11680',
  '층': '7',
  '해제사유발생일': '',
  '해제여부': ''},
 {'거래금액': '197,500',
  '거래유형': '중개거래',
  '건축년도': '2005',
  '년': '2024',
  '동': '',
  '등기일자': '',
  '매도자': '개인',
  '매수자': '개인',
  '법정동': '역삼동',
  '아파트': 'e-편한세상',
  '월': '2',
  '일': '2',
  '전용면적': '59.606',
  '중개사소재지': '서울 강남구',
  '지번': '755-4',
  '지역코드': '11680',
  '층': '25',
  '해제사유발생일': '',
  '해제여부': ''},
 {'거래금액': '69,000',
  '거래유형': '중개거래',
  '건축년도': '2006',
  '년': '2024',
  '동': '',
  '등기일자': '',
  '매도자': '개인',
  '매수자': '개인',
  '법정동': '역삼동',
  '아파트': "역삼I'PARK",
  '월': '2',
  '일': '3',
  '전용면적': '28.246',
  '중개사소재지': '서울 강남구',
  '지번': '713-11',
  '지역코드': '11680',
  '층': '1',
  '해제사유발생일': '',
  '해제여부': ''},
 {'거래금액': '233,000',
  '거래유형': '중개거래',
  '건축년도': '2006',
  '년': '2024',


### 4. Result : Display & Save as file 

In [69]:
result = pd.DataFrame(items_list)

In [82]:
result

,거래금액,거래유형,건축년도,년,동,등기일자,매도자,매수자,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"233,000",중개거래,2006,2024,,,개인,개인,역삼동,역삼푸르지오,2,2,84.9097,서울 강남구,754-1,11680,7,,
1,"197,500",중개거래,2005,2024,,,개인,개인,역삼동,e-편한세상,2,2,59.606,서울 강남구,755-4,11680,25,,
2,"69,000",중개거래,2006,2024,,,개인,개인,역삼동,역삼I'PARK,2,3,28.246,서울 강남구,713-11,11680,1,,
3,"233,000",중개거래,2006,2024,,,개인,개인,역삼동,역삼푸르지오,2,3,84.9097,서울 강남구,754-1,11680,2,,
4,"31,500",중개거래,2002,2024,,24.02.14,개인,개인,역삼동,대우디오빌,2,4,30.03,서울 강남구,720-25,11680,5,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,"40,000",중개거래,2006,2024,,,개인,개인,도곡동,SK허브프리모,2,17,33.8,서울 강남구,953-1,11680,18,,
154,"227,000",중개거래,1983,2024,1,24.03.11,개인,개인,도곡동,삼익,2,19,153.71,서울 강남구,869,11680,5,,
155,"210,000",중개거래,2016,2024,103,24.02.29,법인,개인,도곡동,래미안도곡카운티,2,20,71.85,서울 강남구,538-3,11680,4,,
156,"232,000",중개거래,2016,2024,103,24.04.03,개인,개인,도곡동,래미안도곡카운티,2,23,71.85,서울 강남구,538-3,11680,4,,


In [71]:
result.shape

(158, 19)

In [72]:
result.head()

,거래금액,거래유형,건축년도,년,동,등기일자,매도자,매수자,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"233,000",중개거래,2006,2024,,,개인,개인,역삼동,역삼푸르지오,2,2,84.9097,서울 강남구,754-1,11680,7,,
1,"197,500",중개거래,2005,2024,,,개인,개인,역삼동,e-편한세상,2,2,59.606,서울 강남구,755-4,11680,25,,
2,"69,000",중개거래,2006,2024,,,개인,개인,역삼동,역삼I'PARK,2,3,28.246,서울 강남구,713-11,11680,1,,
3,"233,000",중개거래,2006,2024,,,개인,개인,역삼동,역삼푸르지오,2,3,84.9097,서울 강남구,754-1,11680,2,,
4,"31,500",중개거래,2002,2024,,24.02.14,개인,개인,역삼동,대우디오빌,2,4,30.03,서울 강남구,720-25,11680,5,,


#### : file name

In [73]:
name_info = in_lawd + "_" + in_deal_ymd
name_info

'강남구_202402'

In [74]:
file_name = f"data/result_{name_info}.csv"
file_name

'data/result_강남구_202402.csv'

In [75]:
result.to_csv(file_name, index=False)

## Appendix : Function 

In [76]:
def deal_by_lawd(cd, ymd):
    
    svckey = my_key
    
    area_cd = "LAWD_CD=" + cd
    deal_dt = "DEAL_YMD=" + ymd
    svc_key = "serviceKey=" + svckey
    url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key

    response = requests.get(url)
    items_list = find_xml_items(response)
    result = pd.DataFrame(items_list)
    
    name_info = in_lawd + "_" + in_deal_ymd
    file_name = f"data/result_{name_info}.csv"

    result.to_csv(file_name, index=False)
    
    return file_name

In [77]:
#### - Input Data : LAWD_CD, DEAL_YMD, serviceKey  입력받기
in_lawd = input("검색지역 입력 : ")
print(in_lawd)

in_lawd_cd = find_lawd_cd(in_lawd)
print(in_lawd_cd)

in_deal_ymd = input("계약월 입력(예: 201911) : ")
print(in_deal_ymd)

검색지역 입력 : 강남구
강남구
11680
계약월 입력(예: 201911) : 202402
202402


In [78]:
deal_by_lawd(in_lawd_cd, in_deal_ymd)

'data/result_강남구_202402.csv'